# Deep Learning-Based WWR and Floor Count Extraction from Façade Images to Improve UBEM

CISBAT 2025

[Ayca Duran](https://systems.arch.ethz.ch/ayca-duran), [Panagiotis Karapiperis](https://www.linkedin.com/in/panagiotis-karapiperis-ethz/), [Christoph Waibel](https://systems.arch.ethz.ch/christoph-waibel), [Arno Schlueter](https://systems.arch.ethz.ch/arno-schlueter)

### FCN Windows Finder - Pretrained on COCO with VOC labels

This notebook performs inference of the FCN model (ResNet50) on the rectified images.

In [8]:
# Import Libraries
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

# PyTorch
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models.segmentation import fcn_resnet50

# CPU / GPU
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

## Inference Images Prep

This part of the notebook reads images from the custom inference folder (under example) and prepares them for the trained fcn model.

In [ ]:
images_path = "example/rectified/images"
print(len(os.listdir(images_path)))

In [10]:
# Define transformations for input to the model
to_tensor = transforms.Compose([transforms.Resize((520, 520)), transforms.PILToTensor()])
norm_img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

## Model

Loads the pretrained model from the hub, and loads the weights (output of the fcn_training.ipynb notebook).

In [ ]:
from torchvision.models.segmentation import fcn_resnet50

# Define model name
model_name = "fcn_resnet50" 
trained_model_path = f"trained_models/{model_name}/model_best.pt"

# Load model, adjust classifier and load weights
model = fcn_resnet50(weights='COCO_WITH_VOC_LABELS_V1').to(device)
model.classifier[-1] = nn.Conv2d(model.classifier[4].in_channels,1,kernel_size=(1,1),stride=(1,1)).to(device)
model.aux_classifier[-1] = nn.Conv2d(model.aux_classifier[4].in_channels,1, kernel_size=(1,1),stride=(1,1)).to(device)
model.load_state_dict(torch.load(trained_model_path))

model.eval()
model.to(device)
print(f"{model_name} loaded!")

In [ ]:
# Save path
save_path = f"example/predictions/{model_name}_rectified"
os.makedirs(save_path, exist_ok=True)

# Loop Inference

This part of the notebook iterates over the defined inference images set, and predicts window masks using the trained model (FCN-ResNet50).

In [ ]:
# Iterate
for index, filename in tqdm(enumerate(os.listdir(images_path))):

  # Open image
  image_path = os.path.join(images_path, filename)
  orig_image = Image.open(image_path).convert('RGB')
  size = orig_image.size
  image = to_tensor(orig_image)/255
  img = norm_img(image.type(torch.float))

  # get prediction of windows
  with torch.no_grad():
    prediction = model(img.to(device).unsqueeze(dim=0))['out']
    pred_soft = torch.nn.functional.sigmoid(prediction)
    pred_binary = (pred_soft > 0.5).squeeze().cpu().numpy() # set threshold
  
  mask = pred_binary.astype(int)
  mask = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)

  cv2.imwrite(os.path.join(save_path, filename.split(".jpg")[0]+".png"), mask)
  
  # Optional: Plot
  
  fig, axes = plt.subplots(1,2,figsize=(12,5))
  axes[0].imshow(orig_image)
  axes[0].set_title(f'Original Image {filename}')
  axes[1].imshow(mask, cmap="grey")
  axes[1].set_title('Windows Mask')
  for ax in axes:
      ax.axis(False)
  #break